<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Carga

En este laboratorio, tu objetivo será diseñar la estructura de una base de datos relacional, crear las tablas necesarias y cargar en ellas los datos limpios y preparados que obtuviste en el laboratorio anterior. Trabajarás con datos relacionados con la demanda, generación eléctrica, y variables socioeconómicas, almacenándolos de manera eficiente para facilitar el análisis y las consultas posteriores.


- **Diseño de la Base de Datos:** Define una estructura de base de datos relacional que sea adecuada para almacenar los datos de demanda eléctrica, generación eléctrica, población y PIB por provincia.

- **Creación de la Base de Datos:** Escribe los scripts SQL necesarios para crear las tablas definidas en la estructura de la base de datos. Asegúrate de definir correctamente los tipos de datos y las restricciones (e.g., `NOT NULL`, `UNIQUE`).

- **Carga de Datos en la Base de Datos:** Utiliza Python para cargar los datos limpios y preparados en las tablas de la base de datos. Asegúrate de que los datos se insertan correctamente, manejando posibles errores o inconsistencias en los datos.


In [ ]:
# imports
# ----------------------------------------------------------
import pandas as pd
import psycopg2
from src import soporte_carga as sc
import dotenv
dotenv.load_dotenv()
import os

# Diseño BBDD    

Tabla: generacion_electrica

- fecha: DATE Representa la fecha de generación eléctrica.
- tipo_generacion: VARCHAR Describe el tipo de fuente de energía utilizada (e.g., solar, eólica).
- energia_generada: FLOAT Cantidad de energía generada en unidades específicas (e.g., MW).


Tabla: demanda_electrica

- fecha: DATE Representa la fecha de la demanda eléctrica.
- demanda: FLOAT Valor de la demanda eléctrica en unidades específicas (e.g., MW).


Tabla: datos_economicos

- region: VARCHAR Nombre de la región donde se están tomando los datos económicos.
- year: INT Año al que corresponden los datos económicos.
- pib: FLOAT Producto Interno Bruto (PIB) de la región para el año indicado.
- tasa_paro: FLOAT Tasa de desempleo en la región para el año indicado.


Tabla: datos_demograficos

- region: VARCHAR Nombre de la región de los datos demográficos.
- year: INT Año al que corresponden los datos demográficos.
- poblacion: INT Población total de la región para el año indicado.
- natalidad: FLOAT Tasa de natalidad en la región para el año indicado.

# Creación BBDD

In [ ]:
try:
    dbname = os.getenv("dbname")
    user = os.getenv("postgres")
    password = os.getenv("password")
    host = os.getenv("host")
    
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
    )

    # Crear un cursor para ejecutar comandos SQL
    cursor = connection.cursor()

    # Definir la estructura de las tablas
    query_create_generacion_electrica = """
    CREATE TABLE IF NOT EXISTS generacion_electrica (
        id_registro_generacion SERIAL PRIMARY KEY,
        fecha DATE,
        tipo_generacion VARCHAR(200),
        energia_generada DECIMAL
    )"""
    cursor.execute(query_create_generacion_electrica)

    query_create_demanda_electrica = """
    CREATE TABLE IF NOT EXISTS demanda_electrica (
        id_registro_demanda SERIAL PRIMARY KEY,
        fecha DATE,
        demanda DECIMAL
    )"""
    cursor.execute(query_create_demanda_electrica)

    query_create_datos_economicos = """
    CREATE TABLE IF NOT EXISTS datos_economicos (
        id_registro_economico SERIAL PRIMARY KEY,
        region VARCHAR(200),
        year INT,
        pib DECIMAL,
        tasa_paro DECIMAL
    )"""
    cursor.execute(query_create_datos_economicos)

    query_create_datos_demograficos = """
    CREATE TABLE IF NOT EXISTS datos_demograficos (
        id_registro_demografico SERIAL PRIMARY KEY,
        region VARCHAR(200),
        year INT,
        poblacion INT,
        natalidad DECIMAL
    )"""
    cursor.execute(query_create_datos_demograficos)
    connection.commit()

    # Cerrar la conexión y el cursor
    cursor.close()
    connection.close()
except Exception as e:
    print("Error al conectar a la base de datos:", e)
    raise

# Crear un cursor para ejecutar comandos SQL
cursor = connection.cursor()

# Carga de datos

In [ ]:
# Obtengo los parametros para la conexión a la bbdd
dbname = os.getenv("dbname")
user = os.getenv("postgres")
password = os.getenv("password")
host = os.getenv("host")

# Me conecto a la bbdd
connection = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
)

cursor = connection.cursor()

# Cargo e inserto cada DataFrame en su tabla correspondiente
df_generacion_electrica = pd.read_csv("datos/descargas/ree/generacion_electrica_final.csv")
sc.insertar_datos(cursor, df_generacion_electrica, "generacion_electrica")

df_demanda_electrica = pd.read_csv("datos/descargas/ree/demanda_electrica_final.csv")
sc.insertar_datos(cursor, df_demanda_electrica, "demanda_electrica")

df_datos_demograficos = pd.read_csv("datos/descargas/ine/datos_demograficos.csv")
sc.insertar_datos(cursor, df_datos_demograficos, "datos_economicos")

df_datos_economicos = pd.read_csv("datos/descargas/ine/datos_economicos.csv")
sc.insertar_datos(cursor, df_datos_economicos, "datos_demograficos")

cursor.close()
connection.close()